<a href="https://colab.research.google.com/github/huihuiwuwu/Github-Colab/blob/main/AllData_%ED%86%A0%ED%94%BD_%EB%B6%84%EC%84%9D(AI%2BMedia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI, Media 데이터를 하나로 만듦

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
AI=pd.read_csv('/content/drive/MyDrive/졸업논문/AI Media/AI.csv',usecols=[1])
print(AI[:5])
Media=pd.read_csv('/content/drive/MyDrive/졸업논문/AI Media/media.csv',usecols=[1])
print(Media[:5])
AI.rename(columns={'x':'Contents'})
Media.rename(columns={'x':'Contents'})

In [ ]:
All_data=pd.concat([AI, Media])
print(All_data[:5])
All_data.rename(columns={'x':'Contents'})
All_data.info()
All_data.to_csv('/content/drive/MyDrive/졸업논문/AI Media/All_data.csv', mode='a', index=False, header=True)

# 7월30일: 토픽분석

In [ ]:
import sys
sys.executable

'/usr/bin/python3'

In [ ]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git
from hanspell import spell_checker

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-nfj_5mnr
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-nfj_5mnr
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4810 sha256=503aaf38d469e69abc69b4c82dfd3f938b8bce6183328ec180ce36ba847b879e
  Stored in directory: /tmp/pip-ephem-wheel-cache-c65vhji5/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [ ]:
import site
import os
import pandas as pd
import re
from tqdm import tqdm
from hanspell import spell_checker
from konlpy.tag import Hannanum

In [ ]:
# 토픽 모델링 과정에서 필요한 패키지 설치 및 불러오기

In [ ]:
!pip install pyLDAvis

In [ ]:
import numpy as np
print(np.__version__)

1.25.1


In [ ]:
!pip install nltk

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim import corpora
import numpy as np
import nltk
nltk.download('punkt')
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.gensim_models
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# 데이터 및 사전 불어오기 및 사전 중복 검사 （아래 코드는 실행할 필요가 없음)
def duplicate_check(df, dictionary_name):

    src = list(df)[0]
    dest = list(df)[1]

    dup = df.duplicated([src])
    df_dup = pd.concat([df, dup], axis=1)
    df_dup.rename(columns={0: 'Dup'}, inplace=True)
    df_dup['Dup'].value_counts()

    if df_dup['Dup'].values.sum() > 0:
        print(f"{dictionary_name}에서 중복 key 값이 발견되었습니다!")
        for i in range(len(df_dup)):
            if df_dup['Dup'][i]:
                # print(f"중복되는 key 값 : {df_dup['#en'][i]} (index at {i})")
                print(f"중복되는 key 값 : {df_dup[src][i]} (index at {i})")

    else:
        print(f"{dictionary_name}에서 중복되는 값이 없습니다.")

    print("사전에서 읽어들인 용어의 개수 :", len(df), "\n")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
All_data=pd.read_csv('/content/drive/MyDrive/졸업논문/AI Media/All_data.csv') #AI과 Media 합친 All_data를 불러오기

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
All_data[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,x
0,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...
1,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...
2,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...
3,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...
4,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...


In [ ]:
All_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320049 entries, 0 to 1320048
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   x       1320049 non-null  object
dtypes: object(1)
memory usage: 10.1+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
All_data=All_data.rename(columns={'x':'Contents'}) # column 이름은 "x"를 'Contents'로 바꾸기

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
All_data[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Contents
0,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...
1,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...
2,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...
3,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...
4,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...


In [ ]:
stopwords_df = pd.read_excel("/content/drive/MyDrive/졸업논문/AI Media/stopwords.xlsx")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stopwords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   stopword  1031 non-null   object
dtypes: object(1)
memory usage: 8.2+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
TARGET_COL = "Contents"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
PROCESSED_COL = "ProcessedData"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 데이터의 결측값 여부 확인
print(f'충 데이터(row)의 수:{len(All_data)}')

print(f"\n===== {TARGET_COL} column의 결측값 여부 확인 =====")
print('결측값 여부 :', All_data[TARGET_COL].isnull().values.any())
print(f"원본 데이터의 유니크한 값 : {All_data[TARGET_COL].nunique()}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


충 데이터(row)의 수:1320049

===== Contents column의 결측값 여부 확인 =====
결측값 여부 : False
원본 데이터의 유니크한 값 : 126135


In [ ]:
# 토픽 모델링을 위한 전처리 작업

In [ ]:
All_data[PROCESSED_COL]=[False for _ in range(len(All_data[TARGET_COL]))]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 개행 문자 (CRLF) 제거
def remove_CRLF(input_str):
    input_str = input_str.replace("\n", " ")
    input_str = input_str.replace("\n", " ")
    input_str = input_str.replace("\r", " ")
    input_str = input_str.replace("\r", " ")

    return input_str

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for i in range(len(All_data[TARGET_COL])):
    All_data[PROCESSED_COL][i] = remove_CRLF(All_data[TARGET_COL][i])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  # We already disallow dtype=uint w/ negative numbers


In [ ]:
All_data[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Contents,ProcessedData
0,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...
1,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...
2,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...
3,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...
4,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...


In [ ]:
# 이모티콘 문자들, 사이트 URL주소, 의미없는 문자 지우기
def cleansing(input_str):
    output_str = re.sub(pattern='<[U]\+[A-Z0-9]*>', repl=' ', string=input_str)
    output_str = re.sub(pattern='http[s]?://(?:[a-zA-Z]|[0-9]|[_$\-@\.&+:/?=]|[!*,]|(?:%[0-9a-fA-F_][0-9a-fA-F_]))+', repl=' ', string=output_str)
    output_str = re.sub(pattern='^[0-9]+$', repl=' ', string=output_str)

    return output_str

In [ ]:
for i in range(len(All_data[TARGET_COL])):
    All_data[PROCESSED_COL][i] = cleansing(All_data[PROCESSED_COL][i])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
All_data[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Contents,ProcessedData
0,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...
1,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...
2,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...
3,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...
4,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...


In [ ]:
# 맞춤법 및 띄어쓰기 교정(리뷰로 할 떄 필수 단계로 진행해야 함, 단 시간 오래 걸림. 여기서 뉴스를 사용하기 때문에 이 단계를 승략)
def spell_check(input_str):
    output_str = input_str.replace("&", " ")

    try:
        tmp = spell_checker.check(output_str).checked # 500자가 넘는 경우 빈 문자열로 반환됨
        if tmp!="":
            output_str = tmp
    except:
        print("hanspell : spell_checker error\n")

    return output_str

In [ ]:
print("Spell checking start")

for i in tqdm(range(len(All_data[TARGET_COL]))):
    All_data[PROCESSED_COL][i] = spell_check(All_data[PROCESSED_COL][i])

In [ ]:
# 교정오류 수정 (https://github.com/ni-inha/topic-modeling-of-mom-community/blob/main/topic_modeling_using_LDA.ipynb)

In [ ]:
# 데이터의 앞뒤에 공백 추가
def add_space(input_str):
    return " "+input_str+" "

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for i in range(len(All_data[TARGET_COL])):
    All_data[PROCESSED_COL][i] = add_space(All_data[PROCESSED_COL][i])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
All_data[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Contents,ProcessedData
0,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권...
1,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 ...
2,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제...
3,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터'...
4,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 ...


In [ ]:
# 방법1: 토큰화 전 불용어 제거 (*불어용 사전에사 nursing과 korean, internet를 포함. 본 토픽분석에서 아래 코드를 실행 안 해도 됨)
def remove_stopwords(input_str):
    input_str = re.sub(pattern='([0-9ㄱ-ㅎㅏ-ㅣ]+)', repl='', string=input_str)
    for i in range(len(stopwords_df["nursing"].dropna())):
        input_str = input_str.replace(" "+stopwords_df["nursing"][i]+" ", " ")
    for i in range(len(stopwords_df["korean"].dropna())):
        input_str = input_str.replace(" "+stopwords_df["korean"][i]+" ", " ")
    for i in range(len(stopwords_df["internet"].dropna())):
        input_str = input_str.replace(stopwords_df["internet"][i], " ")
    output_str = re.sub("\s+", " ", input_str)

    return output_str

In [ ]:
for i in tqdm(range(len(All_data[TARGET_COL]))):
    All_data[PROCESSED_COL][i] = remove_stopwords(All_data[PROCESSED_COL][i])

In [ ]:
# 방법2:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stop_words = set(stopwords_df['stopword'].str.lower())  # 假设停用词在'StopWords'列中

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 定义删除停用词的函数
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    return ' '.join(filtered_text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# All_data[PROCESSED_COL] = All_data[PROCESSED_COL].apply(remove_stopwords) # for보다 더 효율적이고 빠른다고 주장함.

In [ ]:
for i in tqdm(range(len(All_data[TARGET_COL]))):
    All_data[PROCESSED_COL][i] = remove_stopwords(All_data[PROCESSED_COL][i])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 1320049/1320049 [11:52<00:00, 1852.16it/s]


In [ ]:
All_data[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Contents,ProcessedData
0,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...,전산망 ‘ 먹통 ’ 사고가 났다 . 지난 30일 강남고속버스터미널 버스표 발권 시스...
1,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...,‘ IT 강국 대한민국 ’ 첨단 IT를 활용한 법률서비스인 ‘ 리걸테크 산업 ’ 은...
2,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...,살아있는 동물처럼 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조하는...
3,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터 '...
4,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...,국내 데이터 네트워크 인공지능 ( AI ) 생태계가 급성장하며 SW AI 역량을 갖...


In [ ]:
All_data.to_csv('/content/drive/MyDrive/졸업논문/AI Media/remove_stopwords.csv',index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#토큰화

In [ ]:
# 토큰화 함수 정의 (mecab를 사용해서 형태소를 토큰화 함)
def tokenize(text):
    tokens = mecab.morphs(text)
    # 길이가 1보다 큰 토큰만 반환
    return [token for token in tokens if len(token) > 1]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for i in tqdm(range(len(All_data[TARGET_COL]))):
    All_data[PROCESSED_COL][i] = tokenize(All_data[PROCESSED_COL][i])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 1320049/1320049 [08:45<00:00, 2512.32it/s]


In [ ]:
All_data[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Contents,ProcessedData
0,전산망 ‘먹통’ 사고가 또 났다. 지난 30일 서울 강남고속버스터미널 버스표 발권 ...,"[전산망, 먹통, 사고, 지난, 30, 강남, 고속버스, 터미널, 버스표, 시스템,..."
1,‘IT 강국 대한민국’이지만 첨단 IT를 활용한 법률서비스인 ‘리걸테크 산업’은 걸...,"[IT, 강국, 대한민국, 첨단, IT, 활용, 법률, 서비스, 리걸, 테크, 산업..."
2,살아있는 동물처럼 네 다리로 걷고 계단과 비탈길에서 자유자재로 움직이는 로봇을 제조...,"[동물, 처럼, 다리, 계단, 비탈길, 에서, 자유자재, 움직이, 로봇, 제조, 보..."
3,대전 동구 역세권지구에 한국기상산업기술원 이전이 가시화되면서 '기상산업 클러스터' ...,"[대전, 동구, 역세, 지구, 한국, 기상, 산업, 기술원, 이전, 가시, 면서, ..."
4,국내 데이터 네트워크 인공지능(AI) 생태계가 급성장하며 SW AI 역량을 갖춘 이...,"[국내, 데이터, 네트워크, 인공지능, AI, 생태, 급성장, SW, AI, 역량,..."


In [ ]:
All_data.to_csv('/content/drive/MyDrive/졸업논문/AI Media/tokenzen_words.csv',index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## 토픽 모델링
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
dictionary = corpora.Dictionary(All_data['ProcessedData'])
dictionary.filter_extremes(no_below=2, no_above=0.5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
corpus = [dictionary.doc2bow(text) for text in All_data['ProcessedData']]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 최적의 토픽 수 계산
coherence_values = []
model_list = []

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    for num_topics in range(start, limit, step):
#         model = pyLDAvis.gensim_models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def find_optimal_number_of_topics(dictionary, corpus, processed_data, limit=8):
    start = 1;
    step = 1;

    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_data, start=start, limit=limit, step=step)

    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')

    dic = { coherence_values:x for x, coherence_values in zip(x, coherence_values) }

    plt.text(dic[max(coherence_values)],min(coherence_values),
             str(dic[max(coherence_values)]) + ' at MAX',
             color='r',
             horizontalalignment='center',
             verticalalignment='bottom')

    plt.show()

    return dic[max(coherence_values)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
All_data['ProcessedData'] = All_data['ProcessedData'].astype(str)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
All_data["tokenized_text"] = All_data.apply(lambda row: nltk.word_tokenize(row[PROCESSED_COL]), axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
tokenized_doc = All_data["tokenized_text"].apply(lambda x: [word for word in x if len(word) > 1])
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
processed_data = tokenized_doc.tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed_data[:5]

In [ ]:
# 벙법 2(실행 안 해도 됨)
def find_optimal_number_of_topics(dictionary, corpus, processed_data):
    limit = 8;
    start = 2;
    step = 1;

    model_list, coherence_values, perpexities_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_data, start=start, limit=limit, step=step)

    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

    x = range(start, limit, step)
    plt.plot(x, perpexities_values)
    plt.xlabel("Num Topics")
    plt.ylabel("perpexities_values")
    plt.legend(("perpexities_values"), loc='best')
    plt.show()

In [ ]:
if __name__ == '__main__':

    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    #최적의 토픽 수 찾기
    find_optimal_number_of_topics(dictionary, corpus, All_data['ProcessedData'])

In [ ]:
limit = 10
NUM_TOPICS = find_optimal_number_of_topics(dictionary, corpus, processed_data, limit)

In [ ]:
j=0
for i in range(len(coherence_values)):
    print(j+2," coherence_value: ",coherence_values[i])
    print(j+2, " perpexities_values: ", perpexities_values[i])
    j+=1

In [ ]:
num_topics = 6

temp = dictionary[0]
id2word = dictionary.id2token

model = model_list[num_topics-2]

top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

In [ ]:
# LDA모델 시작화
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [ ]:
pyLDAvis.save_html(vis, f"results/lda_{NUM_TOPICS}topics.html")